# Задание 6

### Пример входных данных

In [1]:
import pandas as pd
clinton = pd.read_csv("clinton.csv", encoding="utf-8")
trump = pd.read_csv("trump.csv", encoding="utf-8")

In [2]:
trump['text'][0]

'Join me for a 3pm rally - tomorrow at the Mid-America Center in Council Bluffs, Iowa! Tickets:… https://t.co/dfzsbICiXc'

In [3]:
str(trump.loc[0][0])

'Join me for a 3pm rally - tomorrow at the Mid-America Center in Council Bluffs, Iowa! Tickets:… https://t.co/dfzsbICiXc'

In [4]:
clinton.head()

,text
0,The question in this election: Who can put the...
1,"Last night, Donald Trump said not paying taxes..."
2,Couldn't be more proud of @HillaryClinton. Her...
3,"If we stand together, there's nothing we can't..."
4,Both candidates were asked about how they'd co...


## Описание задания

Предлагается написать генератор случайных диалогов на основе твитов Трампа (файл <i>"trump.csv"</i>) и Хеллори Клинтон (файл <i>"clinton.csv"</i>).

* Каждый диалог состоит из нескольких "ходов" (<i>turn</i>).
* Каждый такой <i>turn</i> представляет собой цепочку нескольких реплик разных агентов (участников диалога, их может быть больше двух).
* Цепочка реплик представляет собой набор реплик агентов. Первое сообщение в цепочке - <i>"исходное сообщение"</i> на которое отвечают оставшиеся агенты (по одному сообщению за turn).

Программа должна представлять отдельный пакет следующей структуры:
<ul>
<li>run\_generator.py</li>
<li>dialogsgenerator:
<ul>
<li>agent.py</li>
<li>randomdialog.py</li>
<li>\_\_init__.py</li>
</ul></li>
</ul>

<b>URGENT:</b> в работе ЗАПРЕЩАЕТСЯ использовать циклы:
<ul>
<li>Использование 1 for (или while): max 1 балл</li>
<li>Использование до 4 for (или while): max 0.8 баллов</li>
<li>Использование 4+ for (или while): max 0.5 баллов</li>
</ul>

### Файл agent.py

Описание класса агента (один класс для всех агентов, в нашем случае для Трампа и Клинтон).

In [5]:
from random import randint

In [157]:
from collections import Generator


class Agent(Generator):
    
    def __init__(self, kb, name):
        # Инициализация
        self.tweets_pool = kb['text']
        self.num_tweets = len(kb)
#         print("i'm in init")
        self.name = name

    def send(self, msg):
        # Будем возвращать случайное сообщение из self.tweets_pool
        random_idx = randint(0, self.num_tweets - 1)
#         print("agent name is ", self.name)
        return "{} --> {}".format(self.name, self.tweets_pool[random_idx])

    def throw(self, typ=None, val=None, tb=None):
        # Оставляем как есть
        super().throw(typ, val, tb)

    def __str__(self):
        # Опишем строковое представление класса
        return self.name

    def __repr__(self):
        return str(self)

#### Пример использования:

In [159]:
# from dialogsgenerator import Agent
clint, trum = Agent(clinton, "clinton"), Agent(trump, "trump")
clint.send("hello")
# type(clint)
clint.send("bla bla")

'clinton --> "On the very day Mom was being born in Chicago, Congress was passing the 19th Amendment to the Constitution—giving women the right to vote."'

In [160]:
# print(next(clint))
# print(next(clint))
# print(next(clint))
# print(next(clint))

### Файл randomdialog.py

Описание класса, генерирующего случайный диалог.

In [305]:
import sys
from itertools import chain, repeat, cycle, islice, takewhile

In [376]:
class RandomDialog(object):
    
    def __init__(self, agents, max_len=5):
        # Инициализация
        self.agents = agents
        self.agents_num = len(agents)
        self.max_len = max_len

    def generate(self):
        """
        Генерирует случайный диалог состоящий из 1-max_len цепочек.
        При генерации вызывает функцию turn.
        Возвращаемый объект является генератором.
        """
        
        """
        --------------------- Комментарии к коду: ---------------------------- 
            Мы хотим вернуть (max_len - 1) цепочек ходов, для этого нам нужно (max_len - 1) раз 
            вызвать turn(), но чтобы каждый раз метод turn() возвращал разные диалоги.

             * self_n_times - пары (i, self), где i = 0..max_len-1
             * invoke_turn - вызывает функцию turn у своего второго аргумента (второй аргумент - self)

            Возьмём map от self_n_times, к котором применён invoke_turn, получим то, что нужно            
        """
        
        # пары - (i, self), где i = 1..max_len-1
        self_n_times = zip(range(self.max_len - 1), repeat(self))
        
        # вызывает turn() у второго аргумента
        invoke_turn = lambda args: list(args[1].turn())
        
        # у каждой пары (i, self) вызываем invoke_turn от self
        yield from map(invoke_turn, self_n_times)

    def turn(self):
        """
        Генерирует одну случайную цепочку следующим образом: выбирается случайный агент.
        Он "говорит" случайное сообщение (msg) из своей Agent.kb (используйте next или send(None)).
        (правила того, как выбирать никак не регулируются, вы можете выбирать случайный твит из Agent.kb никак не учитывая
        переданное msg).
        Это сообщение передается с помощью send (помним, что агент - это объект-генератор).
        Далее получаем ответ от всех агентов и возвращаем (генерируем) их (включая исходное сообщение).
        Возвращаемый объект является генератором.
        """
        
        # получаем сообщение от случайного агента
        rand_agent_ind = randint(0, self.agents_num - 1)
        msg = next(self.agents[rand_agent_ind])
        
        # отправляем сообщение всем агентам
        _ = list(map(lambda x: x.send(msg), self.agents))

        # возвращаем ответы всех агентов вместе с исходным сообщением
        it = iter(self.agents)
        yield from chain([msg], list(map(next, it)))

    def eval(self, dialog=None):
        """
        Превращает генератор случайного далога (который возвращается в self.generate())
        в список списков (пример использования далее).
        Возвращаемый объект является списком.
        """
        if dialog is None:
            dialog = self.generate()
                
        return list(dialog)      

    def write(self, dialog=None, target=sys.stdout):
        """
        Записывает результат self.eval() в соответствующий поток.
        """
        
        if dialog is None:
            dialog = self.eval()
        
        sep_turn = lambda turn: "\t" + turn
        
        print_dialogs = lambda enum_turns: list(map(lambda x: print(x, sep="\t", file=target), 
                             chain(["turn {}:".format(enum_turns[0])], 
                                   map(sep_turn, enum_turns[1]))))
        
        list(map(print_dialogs, enumerate(dialog)))
        
    pass


In [377]:
test_random_diaog = RandomDialog([Agent(clinton, "clinton"), Agent(trump, "trump")], max_len=3)

In [378]:
gen = test_random_diaog.turn()
list(gen)
# list(next(gen))

['clinton --> "I\'m here today because I believe in Hillary Clinton. I want you to help elect her as the next President of the United States." —@POTUS',
 "clinton --> Donald Trump's economic plan is a bad deal for working families—and a big tax cut for Donald Trump.\nhttps://t.co/D0oOe6bwza",
 'trump --> Join me tomorrow! #Trump2016 \n#MakeAmericaGreatAgain \n\nOmaha, Nebraska:\nhttps://t.co/2OWQIlNutu\n\nEugene, Oregon:\nhttps://t.co/oroTbvsNdQ']

In [379]:
gen_dialog = test_random_diaog.generate()
list(test_random_diaog.generate())

[["clinton --> Shirley was 5 years old when women won the right to vote. Today, she's rooting for the first woman president. https://t.co/azwiTfH9th",
  'clinton --> "I join Americans in praying for the victims of the attack in Orlando." \n\nHillary\'s statement: https://t.co/MmaGjrSufr',
  "trump --> Crazy @megynkelly supposedly had lyin' Ted Cruz on her show last night. Ted is desperate and his lying is getting worse. Ted can't win!"],
 ["trump --> Lyin' Ted Cruz consistently said that he will, and must, win Indiana. If he doesn't he should drop out of the race-stop wasting time &amp; money",
  'clinton --> Sybrina, you and all mothers of gun violence victims have taught us hate will never win. Thinking of you today. https://t.co/FnsaKtq1hd -H',
  'trump --> Thank you Charlotte, North Carolina!\n#MakeAmericaGreatAgain \nhttps://t.co/Y19nUKkYTc']]

In [380]:
evaluate = test_random_diaog.eval(gen_dialog)
# test_random_diaog.eval(gen_dialog)

In [381]:
test_random_diaog.write(evaluate)

turn 0:
	trump --> Off to Indiana! #Trump2016 https://t.co/zqUdaaSaXD
	clinton --> "There are an awful lot of people who’ve put their trust and faith in Hillary. And she has always delivered.” —@TimKaine
	trump --> Who did the House Task Force on
Urgent Fiscal Issues call- when America needed HELP? 
https://t.co/Q1NErOtW9V
turn 1:
	clinton --> You heard @FLOTUS. Here's how you can get involved in this campaign → https://t.co/WVUvFhb6bp https://t.co/mjEJZ01Lq0
	clinton --> ¡Puerto Rico! Hoy es tu día de ir a votar por Hillary. Comparte esto y déjale saber a tus amigos: #EstoyConElla. https://t.co/0hjWISk8uR
	trump --> Friends in #FL #OH #NC #IL &amp; #MO we would be honored to have your #VOTE! #SuperTuesday #LetsDoThis #MakeAmericaGreatAgain #TrumpTrain 🚂💨💨💨🇺🇸


#### Пример использования:

In [854]:
# from dialogsgenerator import RandomDialog
rd = RandomDialog([Agent(clinton, "clinton"), Agent(trump, "trump")], max_len=2)

In [855]:
generated = rd.generate()
generated

<generator object RandomDialog.generate at 0x0000007DC85901A8>

In [856]:
evaled = rd.eval(generated)
evaled

[["clinton: Let's make it easier for young people to start businesses and fulfill their dreams. https://t.co/cJzNCnaahh https://t.co/vZHW9VZKDT",
  'trump: My list of potential U.S. Supreme Court Justices was very well recieved. During the next number of weeks I may be adding to the list!'],
 ["trump: DON'T LET HILLARY CLINTON DO IT AGAIN!\r #TrumpPence16\r https://t.co/1mGkPNZPKF",
  'clinton: With the first local transmission of Zika in FL, it’s even more critical for Republicans to stop blocking action: https://t.co/KjmWMDOgDl']]

In [857]:
rd.write(evaled)


turn 0:
	clinton: Let's make it easier for young people to start businesses and fulfill their dreams. https://t.co/cJzNCnaahh https://t.co/vZHW9VZKDT
	trump: My list of potential U.S. Supreme Court Justices was very well recieved. During the next number of weeks I may be adding to the list!
turn 1:
 https://t.co/1mGkPNZPKF
	clinton: With the first local transmission of Zika in FL, it’s even more critical for Republicans to stop blocking action: https://t.co/KjmWMDOgDl

### Файл run_generator.py

Содержит функции для генерации и записи нескольких диалогов. Файл должен быть написан так, чтобы его можно было запускать через командную строку:
<img src="cmd.png">
Описание аргументов представлено на рисунке:
<img src="cmd_help.png">
Для разбора аргументов ипользуйте <a href="https://docs.python.org/3/howto/argparse.html">модуль argparse</a>. Все представленные на рисунке аргументы должны быть обработаны (кроме help, он обрабатывается автоматически модулем argparse).

In [846]:
def generate(rd, count_dialogs=5):
    """
    Генерирует count_dialogs диалогов с помощью rd.generate().
    Возвращаемый объект является генератором.
    """
    <your code here>

def write(dialogs, target):
    """
    Записывает сгенерированные диалоги dialogs (это объект-генератор) в поток target.
    """
    <your code here>


if __name__ == "__main__":
    <your code here>


#### Пример использования функций из файла:

In [869]:
import sys
from run_generator import generate, write

In [870]:
dialogs = generate(rd, 2)
dialogs

In [871]:
write(dialogs, sys.stdout)

________________________________________Dialog 0________________________________________
turn 0:
	trump: Crooked Hillary wants to take your 2nd Amendment rights away. Will guns be taken from her heavily armed Secret Service detail? Maybe not!
	clinton: "Bill, that conversation we started in the law library 45 years ago, it is still going strong.” —Hillary
turn 1:
	clinton: Our teachers deserve more than just a pat on the back. They deserve a raise. #TeacherAppreciationDay https://t.co/0sgmUFg6mX
	trump: I visited our Trump Tower campaign headquarters last night, after returning from Ohio and Arizona, and it was packed with great pros - WIN!
________________________________________Dialog 1________________________________________
turn 0:
	clinton: "He won me over with that Mexican rapist speech." —Ann Coulter on Donald Trump https://t.co/GTNOhKYMA2
 #MakeAmericaGreatAgain #Trump2016 https://t.co/7WHS3vbXSw
turn 1:
	trump: .@FreeJesseJames- Just read your complete statement. You are an am